In [1]:
import pandas as pd
import numpy as np
import re
import time

# database interface
import sqlite3

data_path = "C:/Users/SpiffyApple/Documents/USC/Clubs/Cycling"

# Dump current inventory list to a database 

Original file (Current assests.xlsx) is from our Google Drive

## read the data

In [2]:
df = pd.read_excel('/'.join([data_path, 'current assets_ver2.xlsx']), header=1)

## clean and format

In [3]:
# clean prices up
df.loc[:,'price'] = df.price.astype('str').str.replace('\$*','')

In [4]:
# get genders
df.loc[:,'gender'] = ''
df.loc[df.Item.astype('str').str.lower().str.contains(" m$"),'gender'] = 'male'
df.loc[df.Item.astype('str').str.lower().str.contains(" w|women"),'gender'] = 'women'

df.loc[:,'MTB'] = df.Item.str.contains('MTB')

# get retail and team prices
df.loc[:,'retail_price'] = df.price.str.extract('(\d+)',expand=False).astype('float')
teamPrice = df.price.str.contains('\(|\)')

df.loc[:,'team_price'] = df.loc[:,'retail_price']
df.loc[teamPrice,'team_price'] = df.loc[teamPrice,'price'].str.extract('\((\d+)[\w\s]+',expand=False).astype('float')

# team discount
df.loc[:,'team_discount'] = df.team_price<df.retail_price

# get items not for sale
df.loc[:,'not4sale'] = df.retail_price.isnull()

In [5]:
# fetch item types
df.loc[:,'item_type'] = ''

searchStrings = ['jacket','v neck|shirt|tee','jersey','bib|short','hoodie','vest','set','bag','bottle','pedals','stickers','socks','warmers','cad10','gloves','cap']
itemType = ['jacket','tshirt','jersey','bibs','hoodie','vest','wheels','bag','water bottle','pedals','stickers','socks','armwarmers','bike','gloves','cap']

for s,i in zip(searchStrings,itemType):
    sub = df.loc[:,'Item'].astype('str').str.lower().str.contains(s)
    df.loc[sub,'item_type'] = i

## Figure out sizes

In [6]:
# figure out sizes
sizeCols = ['XXS','XS','S','M','L','XL','XXL','XXXL']
df.loc[:,sizeCols] = df.loc[:,sizeCols].replace(np.nan,0)

In [7]:
inventory = df.loc[:,sizeCols].astype('str').apply(lambda s: s.str.replace('\s','').str.lower().str.replace("\(\d\)|scrow|jude|\+",'').str.strip(), axis=1).replace("",0).astype('float').astype('int')
coming = df.loc[:,sizeCols].astype('str').apply(lambda s: s.str.extract('\((\d)',expand=False).str.strip(), axis=1).replace(np.nan,0).astype('int')

In [8]:
# temp for work check
assets = pd.concat([df,inventory,coming],keys = ['original','onhand','coming'],axis=1)
assets.to_csv("/".join([data_path, 'assets.csv']))

In [9]:
df.loc[:,'en_route'] = (coming>0).any(axis=1)

# replace sizes with clean inventory
df.loc[:,sizeCols] = inventory

In [10]:
# fill in missing counts
df.loc[df['count'].isnull(),'count'] = inventory.loc[:,:].sum(axis=1).loc[df['count'].isnull()]

## Dump to database

In [11]:
dbname = "cycling_club_assets.db"
data_path = "/".join([data_path, 'sccycling_code'])
tableName = 'assets'
conn =  sqlite3.connect("/".join([data_path, dbname]))

In [12]:
df.loc[:,'itemID'] = np.arange(df.shape[0])

In [13]:
# drop main inventory file
df.drop(['price'],axis=1).reset_index(drop=True).to_sql(tableName,conn, index=False)

In [14]:
# stuff that's forthcoming

### anything that's coming?
coming.loc[:,'anySize'] = (coming.sum(axis=1)>0)

### link inventory to coming items
coming.loc[:,'itemID'] = df.loc[:,'itemID']
coming.loc[:,'Item'] = df.loc[:,'Item']

### dump to sqlite
coming.loc[coming.anySize>0].to_sql('enRoute',conn, index=False)

In [15]:
# create view between coming and inventory
#cur = conn.cursor()
#cur.execute("CREATE VIEW assets_all AS SELECT * FROM assets INNER JOIN enRoute USING(itemID);")
#conn.commit()

## Tests

In [16]:
pd.read_sql("SELECT * FROM assets WHERE (gender=='male') & (item_type='bibs')", conn)

,Item,Description,count,XXS,XS,S,M,L,XL,XXL,XXXL,gender,MTB,retail_price,team_price,team_discount,not4sale,item_type,en_route,itemID
0,2019 Pedal Mafia Core bib M,None,1.0,0,0,0,1,0,0,0,0,male,0,130.0,72.0,1,0,bibs,1,8
1,2019 Pedal Mafia Tech bib M,None,1.0,0,0,1,0,0,0,0,0,male,0,140.0,105.0,1,0,bibs,1,9
2,2018 Castelli training bib M,None,0.0,0,0,0,0,0,0,0,0,male,0,130.0,79.0,1,0,bibs,0,19
3,2018 Castelli aero bib M,None,2.0,0,0,1,0,0,0,1,0,male,0,140.0,110.0,1,0,bibs,0,20


In [17]:
pd.read_sql('SELECT * FROM enRoute', conn)

,XXS,XS,S,M,L,XL,XXL,XXXL,anySize,itemID,Item
0,0,0,1,0,1,1,1,0,1,4,2019 Pedal Mafia Core Jersey M
1,0,0,0,1,1,1,0,0,1,5,2019 Pedal Mafia Tech Jersey M
2,0,0,0,0,1,0,0,0,1,6,2019 Pedal Mafia Core Jersey W
3,0,0,0,0,1,1,1,0,1,8,2019 Pedal Mafia Core bib M
4,0,0,0,0,2,1,0,0,1,9,2019 Pedal Mafia Tech bib M
5,0,0,1,1,1,0,0,0,1,11,2019 Pedal Mafia MTB Jersey unisex


# Create sales table

In [18]:
out  = pd.read_sql("PRAGMA table_info(assets);",conn)

In [19]:
out.set_index('name',inplace=True)

In [20]:
numericFields = ['XXS','XS','S','M','L','XL','XXL','XXXL','retail_price','team_price']
fields = numericFields + ['itemID','Item','count']

In [21]:
",".join(["%s %s" %(f,t) for f,t in zip(fields,out.loc[fields].type)])

'XXS INTEGER,XS INTEGER,S INTEGER,M INTEGER,L INTEGER,XL INTEGER,XXL INTEGER,XXXL INTEGER,retail_price REAL,team_price REAL,itemID INTEGER,Item TEXT,count REAL'

In [22]:
fields = ",".join(["%s %s" %(f,t) for f,t in zip(fields,out.loc[fields].type)])
fields = fields + ", date TEXT"
query = "CREATE TABLE sales (%s);" %fields
query

'CREATE TABLE sales (XXS INTEGER,XS INTEGER,S INTEGER,M INTEGER,L INTEGER,XL INTEGER,XXL INTEGER,XXXL INTEGER,retail_price REAL,team_price REAL,itemID INTEGER,Item TEXT,count REAL, date TEXT);'

In [23]:
c = conn.cursor()
c.execute(query)

In [24]:
pd.read_sql('SELECT * FROM sales', conn)

,XXS,XS,S,M,L,XL,XXL,XXXL,retail_price,team_price,itemID,Item,count,date


In [25]:
conn.close()